# Solving linear equations by different methods

**Randall Romero Aguilar, PhD**

This demo is based on the original Matlab demo accompanying the  <a href="https://mitpress.mit.edu/books/applied-computational-economics-and-finance">Computational Economics and Finance</a> 2001 textbook by Mario Miranda and Paul Fackler.

Original (Matlab) CompEcon file: **demlin01.m**

Running this file requires the Python version of CompEcon. This can be installed with pip by running

    !pip install compecon --upgrade

<i>Last updated: 2022-Ago-07</i>
<hr>

    

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import solve, inv
from timeit import default_timer as timer
from numba import jit

Make a function to time 

In [2]:
tic = lambda: timer()
toc = lambda t: 1000* (timer() - t)  # ellapsed milliseconds

## Milliseconds required to solve n by n linear equation $Ax = b$
m times using solve(A, b) and dot(inv(A), b), computing inverse only once.

In [3]:
mvalues = [1, 100]
nvalues = [50, 500]

cases = pd.MultiIndex.from_product([mvalues, nvalues], names=['m','n'])
results0 = pd.DataFrame(index=cases, columns=['solve(A,b)', 'inv(A) @ b'])

for m, n in cases:
    A = np.random.rand(n, n)
    b = np.random.rand(n, 1)

    tt = tic()
    for j in range(m):
        x = solve(A, b)

    results0.loc[(m, n), 'solve(A,b)'] = toc(tt)

    tt = tic()
    Ainv = inv(A)
    for j in range(m):
        x = Ainv @ b

    results0.loc[(m, n), 'inv(A) @ b'] = toc(tt)

In [4]:
@jit
def using_solve(A, b, m):
    for j in range(m):
        x = solve(A, b)

@jit
def using_inv(Ainv, b, m):    
    for j in range(m):
        x = Ainv @ b

#run once to compile
using_solve(A, b, m)
using_inv(Ainv, b, m)

In [5]:
results1 = pd.DataFrame(index=cases, columns=['solve(A,b)', 'inv(A) @ b'])

for m, n in cases:
    A = np.random.rand(n, n)
    b = np.random.rand(n, 1)

    tt = tic()
    using_solve(A, b, m)

    results1.loc[(m, n), 'solve(A,b)'] = toc(tt)

    tt = tic()
    Ainv = inv(A)
    using_inv(Ainv, b, m)

    results1.loc[(m, n), 'inv(A) @ b'] = toc(tt)

In [6]:
pd.concat([results0, results1], keys=['without jit', 'using jit'], axis=1).style.highlight_min(axis=1)